#### This notebook evaluates the identity model on the Norman dataset.

The identity model (= "no effect model") assumes that perturbations have no effect and predictions thus correspond to control cells. These 'predictions' are computed on-the-fly in this notebook (rather than being loaded as in the other baseline models).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import jax
import os
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp
import anndata as ad
import pandas as pd
from tqdm.auto import tqdm

In [3]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]

In [4]:
split = 4

#### Data loading

In [5]:
DATA_DIR = "/home/haicu/soeren.becker/repos/ot_pert_reproducibility/norman2019/norman_preprocessed_adata"

adata_train_path = os.path.join(DATA_DIR, f"adata_train_pca_3_split_{split}.h5ad")
adata_test_path = os.path.join(DATA_DIR, f"adata_val_pca_3_split_{split}.h5ad")
adata_ood_path = os.path.join(DATA_DIR, f"adata_test_pca_3_split_{split}.h5ad")

# load data splits
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

#### Add subgroup annotations

- single: Only one gene has been perturbed. This perturbation is not part of the training data.
- double_seen_0: Two genes have been perturbed. Neither of these gene perturbations have been seen as single perturbations in the training data.
- double_seen_1: Two genes have been perturbed. One of the two perturbed genes has been seen as single perturbation in the training data.
- double_seen_2: Two genes have been perturbed. Both gene perturbations have been seen as single perturbation in the training data.

In [6]:
train_conditions = adata_train.obs.condition.str.replace("+ctrl", "").str.replace("ctrl+", "").unique()

assert not adata_ood[adata_ood.obs.condition != "ctrl"].obs.condition.isin(train_conditions).any()

mask_single_perturbation = adata_ood.obs.condition.str.contains("ctrl")
mask_double_perturbation_seen_0 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    ~adata_ood.obs.gene_1.isin(train_conditions) & 
    ~adata_ood.obs.gene_2.isin(train_conditions)
)
mask_double_perturbation_seen_1 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    (
        (adata_ood.obs.gene_1.isin(train_conditions) & ~adata_ood.obs.gene_2.isin(train_conditions)) | 
        (~adata_ood.obs.gene_1.isin(train_conditions) & adata_ood.obs.gene_2.isin(train_conditions))
    )
)
mask_double_perturbation_seen_2 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    adata_ood.obs.gene_1.isin(train_conditions) & 
    adata_ood.obs.gene_2.isin(train_conditions)
)
adata_ood.obs.loc[mask_single_perturbation, "subgroup"] = "single"
adata_ood.obs.loc[mask_double_perturbation_seen_0, "subgroup"] = "double_seen_0"
adata_ood.obs.loc[mask_double_perturbation_seen_1, "subgroup"] = "double_seen_1"
adata_ood.obs.loc[mask_double_perturbation_seen_2, "subgroup"] = "double_seen_2"

display(adata_ood.obs.subgroup.value_counts())

subgroup
double_seen_1    14554
single           13520
double_seen_2     3934
double_seen_0     1587
Name: count, dtype: int64

In [ ]:
# compute pca on full dataset
adata_all = ad.concat((adata_train, adata_test, adata_ood))
cfpp.centered_pca(adata_all, n_comps=10)

/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


#### Predict on test set

In [ ]:
# compute predictions of identity model --> the identity model always predicts that the perturbation has not effect, 
# i.e., predictions correspond to cells of the control condition
adata_pred_test = adata_test[adata_test.obs["condition"]=="ctrl"]

# project predictions and ground truth data onto pca space
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_all)
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_all)

# dict for ground truths per condition, encoded (=pca space) and decoded (= gene space)
test_data_target_encoded, test_data_target_decoded = {}, {}

# dict for predictions per condition, encoded (=pca space) and decoded (= gene space)
test_data_target_encoded_predicted, test_data_target_decoded_predicted = {}, {}

for cond in adata_test.obs["condition"].cat.categories:
    if cond == "ctrl":
        continue

    # pca space
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_encoded_predicted[cond] = adata_pred_test.obsm["X_pca"]
    
    # gene space
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X
    test_data_target_decoded_predicted[cond] = adata_pred_test.X

#### Predict on ood set (full ood set + subgroups)

In [ ]:
# compute predictions of identity model --> the identity model always predicts that the perturbation has not effect, 
# i.e., predictions correspond to cells of the control condition
adata_pred_ood = adata_ood[adata_ood.obs["condition"]=="ctrl"]

cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_all)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_all)

ood_data_target_encoded, ood_data_target_decoded = {}, {}
ood_data_target_encoded_predicted, ood_data_target_decoded_predicted = {}, {}

subgroups = ["all", "single", "double_seen_0", "double_seen_1", "double_seen_2"]

for subgroup in subgroups:

    ood_data_target_encoded[subgroup] = {}
    ood_data_target_decoded[subgroup] = {}
    ood_data_target_encoded_predicted[subgroup] = {}
    ood_data_target_decoded_predicted[subgroup] = {}
    
    for cond in adata_ood.obs["condition"].cat.categories:
        if cond == "ctrl":
            continue
        
        select = adata_ood.obs["condition"] == cond
        if subgroup != "all":
            select = select & (adata_ood.obs.subgroup == subgroup)

        if not any(select):
            # the condition is not part of this subgroup
            continue
        
        # pca space
        ood_data_target_encoded[subgroup][cond] = adata_ood[select].obsm["X_pca"]
        ood_data_target_encoded_predicted[subgroup][cond] = adata_pred_ood.obsm["X_pca"]
    
        # gene space
        ood_data_target_decoded[subgroup][cond] = adata_ood[select].X
        ood_data_target_decoded_predicted[subgroup][cond] = adata_pred_ood.X

#### Evaluation on test set

In [10]:
# test set: evaluation in encoded (=pca) space
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, 
    # compute_metrics_fast, 
    test_data_target_encoded, 
    test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(
    test_metrics_encoded, 
    prefix="encoded_test_"
)

# test set: evaluation in decoded (=gene) space
test_metrics_decoded = jax.tree_util.tree_map(
    # compute_metrics, 
    compute_metrics_fast, 
    test_data_target_decoded, 
    test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(
    test_metrics_decoded, 
    prefix="decoded_test_"
)

# test set
test_deg_dict = {
    k: v 
    for k, v in adata_train.uns['rank_genes_groups_cov_all'].items() 
    if k in test_data_target_decoded_predicted.keys()
}

# test set
test_deg_target_decoded_predicted = jax.tree_util.tree_map(
    get_mask, 
    test_data_target_decoded_predicted, 
    test_deg_dict
)
test_deg_target_decoded = jax.tree_util.tree_map(
    get_mask, 
    test_data_target_decoded, 
    test_deg_dict
)

deg_test_metrics = jax.tree_util.tree_map(
    compute_metrics, 
    # compute_metrics_fast, 
    test_deg_target_decoded, 
    test_deg_target_decoded_predicted
)
deg_mean_test_metrics = compute_mean_metrics(
    deg_test_metrics, 
    prefix="deg_test_"
)

In [11]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -1.475508607667068,
 'encoded_test_sinkhorn_div_1': 19.92545300516589,
 'encoded_test_sinkhorn_div_10': 15.543769244489999,
 'encoded_test_sinkhorn_div_100': 12.902872282883216,
 'encoded_test_e_distance': 24.981702142470645,
 'encoded_test_mmd': 0.062432351971751655}

In [12]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9534846749799005,
 'decoded_test_sinkhorn_div_1': 121.61307630867793,
 'decoded_test_sinkhorn_div_10': 88.21131173495588,
 'decoded_test_sinkhorn_div_100': 18.533345715752965,
 'decoded_test_e_distance': 33.690288395935475,
 'decoded_test_mmd': 0.028633100799573905}

In [13]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.8073588198628919,
 'deg_test_sinkhorn_div_1': 24.110808273841595,
 'deg_test_sinkhorn_div_10': 13.296171056813208,
 'deg_test_sinkhorn_div_100': 11.819062693365689,
 'deg_test_e_distance': 23.36795391162108,
 'deg_test_mmd': 0.06250722134678528}

#### Evaluation on ood set

In [14]:
ood_metrics_encoded, mean_ood_metrics_encoded = {}, {}
ood_metrics_decoded, mean_ood_metrics_decoded = {}, {}
deg_ood_metrics, deg_mean_ood_metrics = {}, {}
ood_deg_dict = {}
ood_deg_target_decoded_predicted, ood_deg_target_decoded = {}, {}

for subgroup in tqdm(subgroups[::-1]):

    print(f"subgroup: {subgroup}")

    print("Computing ood_metrics_encoded")
    # ood set: evaluation in encoded (=pca) space
    ood_metrics_encoded[subgroup] = jax.tree_util.tree_map(
        compute_metrics, 
        # compute_metrics_fast, 
        ood_data_target_encoded[subgroup], 
        ood_data_target_encoded_predicted[subgroup]
    )
    mean_ood_metrics_encoded[subgroup] = compute_mean_metrics(
        ood_metrics_encoded[subgroup], 
        prefix="encoded_ood_",
    )

    print("Computing ood_metrics_decoded")
    # ood set: evaluation in decoded (=gene) space
    ood_metrics_decoded[subgroup] = jax.tree_util.tree_map(
        # compute_metrics, 
        compute_metrics_fast, 
        ood_data_target_decoded[subgroup], 
        ood_data_target_decoded_predicted[subgroup]
    )
    mean_ood_metrics_decoded[subgroup] = compute_mean_metrics(
        ood_metrics_decoded[subgroup], 
        prefix="decoded_ood_",
    )

    # ood set
    ood_deg_dict[subgroup] = {
        k: v
        for k, v in adata_train.uns['rank_genes_groups_cov_all'].items() 
        if k in ood_data_target_decoded_predicted[subgroup].keys()
    }

    print("Apply DEG mask")
    # ood set
    ood_deg_target_decoded_predicted[subgroup] = jax.tree_util.tree_map(
        get_mask, 
        ood_data_target_decoded_predicted[subgroup], 
        ood_deg_dict[subgroup]
    )
    
    ood_deg_target_decoded[subgroup] = jax.tree_util.tree_map(
        get_mask, 
        ood_data_target_decoded[subgroup], 
        ood_deg_dict[subgroup]
    )

    print("Compute metrics on DEG subsetted decoded")
    deg_ood_metrics[subgroup] = jax.tree_util.tree_map(
        compute_metrics, 
        # compute_metrics_fast, 
        ood_deg_target_decoded[subgroup], 
        ood_deg_target_decoded_predicted[subgroup]
    )
    deg_mean_ood_metrics[subgroup] = compute_mean_metrics(
        deg_ood_metrics[subgroup], 
        prefix="deg_ood_"
    )

  0%|          | 0/5 [00:00<?, ?it/s]

subgroup: double_seen_2
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: double_seen_1
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: double_seen_0
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: single
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: all
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded


In [15]:
mean_ood_metrics_encoded

{'double_seen_2': {'encoded_ood_r_squared': -1.3311141545955951,
  'encoded_ood_sinkhorn_div_1': 25.029296728280876,
  'encoded_ood_sinkhorn_div_10': 20.358269911545975,
  'encoded_ood_sinkhorn_div_100': 17.14217934241662,
  'encoded_ood_e_distance': 33.13209023931311,
  'encoded_ood_mmd': 0.07273576902942015},
 'double_seen_1': {'encoded_ood_r_squared': -0.8474068739197471,
  'encoded_ood_sinkhorn_div_1': 33.39131384762851,
  'encoded_ood_sinkhorn_div_10': 28.565689399025658,
  'encoded_ood_sinkhorn_div_100': 24.348680218783294,
  'encoded_ood_e_distance': 46.590678007595145,
  'encoded_ood_mmd': 0.09871249649335037},
 'double_seen_0': {'encoded_ood_r_squared': -0.8229820330937704,
  'encoded_ood_sinkhorn_div_1': 50.950188318888344,
  'encoded_ood_sinkhorn_div_10': 45.37500402662489,
  'encoded_ood_sinkhorn_div_100': 37.9834172990587,
  'encoded_ood_e_distance': 71.01400564634683,
  'encoded_ood_mmd': 0.131231975224283},
 'single': {'encoded_ood_r_squared': -0.8406988546252251,
  'enc

In [16]:
mean_ood_metrics_decoded

{'double_seen_2': {'decoded_ood_r_squared': 0.9388133883476257,
  'decoded_ood_sinkhorn_div_1': 130.41300729604868,
  'decoded_ood_sinkhorn_div_10': 96.71006686870868,
  'decoded_ood_sinkhorn_div_100': 24.0595462505634,
  'decoded_ood_e_distance': 44.58967001081147,
  'decoded_ood_mmd': 0.03440484332923706},
 'double_seen_1': {'decoded_ood_r_squared': 0.9246253869750283,
  'decoded_ood_sinkhorn_div_1': 138.23160081343218,
  'decoded_ood_sinkhorn_div_10': 105.03367177789862,
  'decoded_ood_sinkhorn_div_100': 30.67837732488459,
  'decoded_ood_e_distance': 55.753075031650326,
  'decoded_ood_mmd': 0.04153280451216481},
 'double_seen_0': {'decoded_ood_r_squared': 0.8854311307271322,
  'decoded_ood_sinkhorn_div_1': 166.538330078125,
  'decoded_ood_sinkhorn_div_10': 134.64654286702475,
  'decoded_ood_sinkhorn_div_100': 46.24297417534722,
  'decoded_ood_e_distance': 81.1451667621896,
  'decoded_ood_mmd': 0.05255092970199055},
 'single': {'decoded_ood_r_squared': 0.9597026377916336,
  'decoded_

In [17]:
deg_mean_ood_metrics

{'double_seen_2': {'deg_ood_r_squared': 0.7362335324287415,
  'deg_ood_sinkhorn_div_1': 28.627122292151817,
  'deg_ood_sinkhorn_div_10': 16.702361473670372,
  'deg_ood_sinkhorn_div_100': 14.97286106989934,
  'deg_ood_e_distance': 29.619239344810318,
  'deg_ood_mmd': 0.07369437469885899},
 'double_seen_1': {'deg_ood_r_squared': 0.7059335784478621,
  'deg_ood_sinkhorn_div_1': 32.59614874341271,
  'deg_ood_sinkhorn_div_10': 20.877838969230652,
  'deg_ood_sinkhorn_div_100': 18.555315350402484,
  'deg_ood_e_distance': 36.54178002599698,
  'deg_ood_mmd': 0.09210377755490216},
 'double_seen_0': {'deg_ood_r_squared': 0.5911642842822604,
  'deg_ood_sinkhorn_div_1': 39.88064320882162,
  'deg_ood_sinkhorn_div_10': 27.592570039961075,
  'deg_ood_sinkhorn_div_100': 23.456626574198406,
  'deg_ood_e_distance': 45.664735144740476,
  'deg_ood_mmd': 0.11046814007891549},
 'single': {'deg_ood_r_squared': 0.8534903779625893,
  'deg_ood_sinkhorn_div_1': 20.89128379225731,
  'deg_ood_sinkhorn_div_10': 11.95

In [18]:
collected_results = {
    # test
    "test_metrics_encoded": test_metrics_encoded,
    "mean_test_metrics_encoded": mean_test_metrics_encoded,
    "test_metrics_decoded": test_metrics_decoded, 
    "mean_test_metrics_decoded": mean_test_metrics_decoded,
    "test_deg_dict": test_deg_dict,
    "test_deg_target_decoded_predicted": test_deg_target_decoded_predicted,
    "test_deg_target_decoded": test_deg_target_decoded,
    "deg_test_metrics": deg_test_metrics,
    "deg_mean_test_metrics": deg_mean_test_metrics,
    # ood
    "ood_metrics_encoded": ood_metrics_encoded,
    "mean_ood_metrics_encoded": mean_ood_metrics_encoded,
    "ood_metrics_decoded": ood_metrics_decoded,
    "mean_ood_metrics_decoded": mean_ood_metrics_decoded,
    "deg_ood_metrics": deg_ood_metrics,
    "deg_mean_ood_metrics": deg_mean_ood_metrics,
    "ood_deg_dict": ood_deg_dict,
    "ood_deg_target_decoded_predicted": ood_deg_target_decoded_predicted,
    "ood_deg_target_decoded": ood_deg_target_decoded,
}

In [19]:
OUT_DIR = "/lustre/groups/ml01/workspace/ot_perturbation/data/norman_soren/identity"
pd.to_pickle(collected_results, os.path.join(OUT_DIR, f"norman_identity_split_{split}_collected_results.pkl"))